In [3]:
import pandas as pd
import os
import glob
import numpy as np

In [4]:
# This is a script to obtain lexical features from the output of the audio - asr transcript - prompt alignment.

In [5]:
basePath = '/vol/tensusers2/wharmsen/JASMIN-fluency-features/comp-q-read_nl_age7-11_nat/04_asr'
asrSystem = 'whispert'

# alignmentTypes = ['csv-align-forward', 'csv-align-reversed']
alignmentTypes = ['csv-align-forward']


for alignmentType in alignmentTypes:
    
    fileList = glob.glob(os.path.join(basePath, asrSystem + '/' + alignmentTypes[0] + '/' + '*.csv' ))
    
    outputDictList = []
    for file in fileList:
        basename = os.path.basename(file).replace('.csv', '')

        outputDict = {}
        
        # Read DF and rename columns
        df = pd.read_csv(file, index_col=0).rename(columns = {'prompt_label' : 'label', 'prompt_conf': 'start', 'prompt_start': 'end', 'prompt_end': 'conf', 'prompt_miscue': 'miscue'})

        # Get nr of correct and incorrect prompts
        # Get correct and incorrect words
        try:
            nr_correct = df['correct'].value_counts()[True]
        except:
            nr_correct = 0 

        try:
            nr_incorrect = df['correct'].value_counts()[False]
        except:
            nr_incorrect = 0 

        df.loc[0,:]

        nrPrompts = len(df)
        outputDict['nr_correct'] = nr_correct
        outputDict['nr_incorrect'] = nr_incorrect
        outputDict['nr_prompts'] = nrPrompts

        # get nr of deletions, substitutions, correct readings and other
        miscue_types = df['miscue'].value_counts().keys()
        for miscue_type in miscue_types:
            if miscue_type in ['del', 'sub', 'cor']: 
                outputDict[miscue_type] = df['miscue'].value_counts()[miscue_type]
            else:
                outputDict['other'] = len(df[~df['miscue'].isin(['del', 'sub', 'cor'])])

        # Read insertion file and get nr of insertions
        ins_file = file.replace('reversed', 'reversed-ins').replace('forward', 'forward-ins')
        insDF = pd.read_csv(ins_file, index_col=0)

        outputDict['ins'] = len(insDF)

        # nr_sub = df['miscue'].value_counts()['sub']
        # nr_del = df['miscue'].value_counts()['del']
        # nr_cor = df['miscue'].value_counts()['cor']
        # nr_other = len(df[~df['miscue'].isin(['del', 'sub', 'cor'])])
        # outputDict['other_miscues'] = df[~df['miscue'].isin(['del', 'sub', 'cor'])]['miscue']
        # print(list(df[~df['miscue'].isin(['del', 'sub', 'cor'])]['label']))
        outputDictList.append({basename: outputDict})

    outputDF = pd.concat([pd.DataFrame.from_dict(x, orient='index') for x in outputDictList])

    outputDF['sub_perc'] = outputDF['sub']/outputDF['nr_prompts']
    outputDF['del_perc'] = outputDF['del']/outputDF['nr_prompts']
    outputDF['ins_perc'] = outputDF['ins']/outputDF['nr_prompts']
    outputDF['cor_perc'] = outputDF['cor']/outputDF['nr_prompts']     
    outputDF['cor_prompt_perc'] = outputDF['nr_correct']/outputDF['nr_prompts']        

outputDF

,nr_correct,nr_incorrect,nr_prompts,cor,sub,del,ins,sub_perc,del_perc,ins_perc,cor_perc,cor_prompt_perc
N000025-AVI6_story1,172,6,178,172,3,3.0,2,0.016854,0.016854,0.011236,0.966292,0.966292
N000025-AVI6_story2,415,46,461,414,43,4.0,15,0.093275,0.008677,0.032538,0.898048,0.900217
N000055-AVI3_story2,379,35,414,368,34,12.0,10,0.082126,0.028986,0.024155,0.888889,0.915459
N000026-AVI5_story2,413,30,443,409,27,7.0,9,0.060948,0.015801,0.020316,0.923251,0.932280
N000027-AVI4_story1,140,9,149,140,7,2.0,3,0.046980,0.013423,0.020134,0.939597,0.939597
...,...,...,...,...,...,...,...,...,...,...,...,...
N000213-AVI6_story2,420,41,461,419,33,9.0,6,0.071584,0.019523,0.013015,0.908894,0.911063
N000214-AVI9_story1,216,8,224,215,7,2.0,5,0.031250,0.008929,0.022321,0.959821,0.964286
N000214-AVI9_story2,385,472,857,377,45,435.0,12,0.052509,0.507585,0.014002,0.439907,0.449242
N000215-AVI8_story1,179,11,190,176,10,4.0,1,0.052632,0.021053,0.005263,0.926316,0.942105


In [19]:
start = df.iloc[0]['start']
# idx = len(df)-1
# print(idx)
end = df.iloc[len(df)-1]['end']

duration = end - start
print(duration)

117.96


In [19]:
dict1 = {'file1': {'A1': 'a', 'B1':'b'}}
dict2 = {'file2' : {'A1': 'a', 'C1':'c'}}
pd.concat([pd.DataFrame.from_dict(dict1, orient='index'), pd.DataFrame.from_dict(dict2, orient='index')])

,A1,B1,C1
file1,a,b,NaN
file2,a,NaN,c


In [6]:
# Input files
basePath = '/vol/tensusers2/wharmsen/JASMIN-fluency-features/comp-q-read_nl_age7-11_nat/04_asr'
asrSystem = 'whispert'
alignmentTypes = ['csv-align-forward', 'csv-align-reversed']
filename = 'N000025-AVI6_story1.csv'

df_file = os.path.join(basePath, asrSystem + '/' + alignmentTypes[0] + '/' + filename)

df = pd.read_csv(df_file, index_col=0).rename(columns = {'prompt_conf': 'start', 'prompt_start': 'end', 'prompt_end': 'conf', 'prompt_miscue': 'miscue'})
df

,index,prompt,aligned_ref,aligned_asrTrans,correct,prompt_label,start,end,conf,miscue
promptID,,,,,,,,,,
0-0-jeroen,0,jeroen,jeroen,jeroen,True,jeroen,0.12,0.80,0.791,cor
0-1-is,1,is,is,is,True,is,0.80,0.96,0.906,cor
0-2-onze,2,onze,onze,onze,True,onze,0.96,1.40,0.865,cor
0-3-buurman,3,buurman,buurman,buurman,True,buurman,1.40,2.12,0.934,cor
1-0-hij,4,hij,hij,hij,True,hij,2.56,2.74,0.897,cor
...,...,...,...,...,...,...,...,...,...,...
20-3-haar,173,haar,haar,haar,True,haar,116.12,116.32,0.909,cor
20-4-vaak,174,vaak,vaak,vaak,True,vaak,116.32,116.82,0.636,cor
20-5-genoeg,175,genoeg,genoeg,genoeg,True,genoeg,116.82,117.30,0.954,cor


In [21]:
# Get correct and incorrect words
nr_correct = df['correct'].value_counts()[True]
nr_incorrect = df['correct'].value_counts()[False]
print(nr_correct, nr_incorrect)

171 7


In [22]:
df['miscue'].value_counts()

miscue
cor    170
sub      6
del      2
Name: count, dtype: int64

In [23]:
# Get lexical features
list_with_unique_miscues = list(df['miscue'].value_counts().keys())
list_with_unique_miscues[0], df['miscue'].value_counts()[list_with_unique_miscues[0]]

('cor', 170)

In [24]:
# Sentence-level measures

# 1. Inter-sentential and intra-sentential pauses

df['sentence_nr'] = [x.split('-')[0] for x in df.index]
df

,index,prompt,aligned_ref,aligned_asrTrans,reversed_aligned_ref,reversed_aligned_asrTrans,correct,prompt_label,start,end,conf,miscue,sentence_nr
promptID,,,,,,,,,,,,,
0-0-jeroen,0,jeroen,jeroen,jeroen,jeroen,jeroen,True,jeroen,0.12,0.80,0.791,cor,0
0-1-is,1,is,is,is,is,is,True,is,0.80,0.96,0.906,cor,0
0-2-onze,2,onze,onze,onze,onze,onze,True,onze,0.96,1.40,0.865,cor,0
0-3-buurman,3,buurman,buurman,buurman,buurman,buurman,True,buurman,1.40,2.12,0.934,cor,0
1-0-hij,4,hij,hij,hij,hij,hij,True,hij,2.56,2.74,0.897,cor,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20-3-haar,4,haar,haar,haar,haar,haar,True,haar,116.12,116.32,0.909,cor,20
20-4-vaak,5,vaak,vaak,vaak,vaak,vaak,True,vaak,116.32,116.82,0.636,cor,20
20-5-genoeg,6,genoeg,genoeg,genoeg,genoeg,genoeg,True,genoeg,116.82,117.30,0.954,cor,20


In [25]:
def getDescriptiveStatistics(scores):
    scores_dict = pd.Series(scores).describe().to_dict()
    try:
        scores_dict['IQR'] = round(scores_dict['75%'] - scores_dict['25%'],3)
    except:
        scores_dict['IQR'] = np.nan
    return scores_dict

def getIntrasententialPauses(sentenceDF):

    if len(sentenceDF)>0:

        pauses_end = list(sentenceDF['start'])[1:] # remove first start time, which represents end of initial pause
        pauses_start = list(sentenceDF['end'])[:-1] # remove last end time, which represents start of final pause
        
        # Get durations of pauses
        pauses_durations = np.array(pauses_end)-np.array(pauses_start)

        # Remove pauses with a duration < 0.2s (=200ms)
        pauses_durations_without0 = [pause_dur for pause_dur in pauses_durations if pause_dur >= 0.2]

        return [round(x,4) for x in pauses_durations_without0 if x > 0.0]
    
    else:
        return []

In [29]:
intrasentential_pause_list = []
for sentence_nr in set(df['sentence_nr']):
    
    # Get rows of DF with selected sentence number
    sentenceDF = df[df['sentence_nr'] == sentence_nr].reset_index()
    begin_time_sentence = sentenceDF.loc[0,'start']
    end_time_sentence = sentenceDF.loc[len(sentenceDF)-1,'start']

    intrasentential_pause_list.append(getIntrasententialPauses(sentenceDF))

intersentential_pauses = [item for sublist in intrasentential_pause_list for item in sublist]
print(intersentential_pauses)    
output = getDescriptiveStatistics(intersentential_pauses)
output

[0.36, 0.86, 0.38, 0.68, 0.3]


{'count': 5.0,
 'mean': 0.516,
 'std': 0.2422395508582362,
 'min': 0.3,
 '25%': 0.36,
 '50%': 0.38,
 '75%': 0.68,
 'max': 0.86,
 'IQR': 0.32}

In [31]:
d=[{'dict1': {'count': 5.0,
 'mean': 0.516,
 'std': 0.2422395508582362,
 'min': 0.3,
 '25%': 0.36,
 '50%': 0.38,
 '75%': 0.68,
 'max': 0.86,
 'IQR': 0.32}},
 {'dict2' : {'count': 5.0,
 'mean': 0.516,
 'std': 0.2422395508582362,
 'min': 0.3,
 '25%': 0.36,
 '50%': 0.38,
 '75%': 0.68,
 'max': 0.86,
 'IQR': 0.32}}]

pd.concat(d)

TypeError: cannot concatenate object of type '<class 'dict'>'; only Series and DataFrame objs are valid

In [27]:
# Create sentence-word identifier
df['sentence-word-idx'] = [x.split('-')[0] + '-' + x.split('-')[1] for x in df.index]

# Select only first and last word of each sentence
first_last_word_id_list= []
for sentence_nr in set(df['sentence_nr']):
    sentenceDF = df[df['sentence_nr'] == sentence_nr].reset_index()
    first_word_id = 0
    last_word_id = len(sentenceDF) -1
    first_last_word_id_list.append(str(sentence_nr) + '-' + str(first_word_id))
    first_last_word_id_list.append(str(sentence_nr) + '-' + str(last_word_id))

interPauseDF = df[df['sentence-word-idx'].isin(first_last_word_id_list)]

getIntrasententialPauses(interPauseDF)[1::2]


[0.44,
 0.34,
 3.08,
 0.98,
 0.92,
 2.78,
 0.74,
 1.36,
 5.3,
 0.9,
 0.72,
 6.48,
 0.84,
 0.38,
 5.12,
 0.42,
 4.54,
 1.14,
 1.22,
 0.76]

In [ ]:
interPauseDF